In [11]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
data={"intents":[
    {
      "tag":"greeting",
      "patterns":["Hi","Hello","Hey","How are u?"],
      "responses":["How are you doing?","Hello","Greetings!","Howdy partner!"]
    },
    {
        "tag":"age",
        "patterns":["how old are you","when were you born","when is your birthday","how old are u"],
        "responses":["I am 20 years old","I was born in 2002","My birthday is June 29th and I was born in 2002","29/06/2002"]
    },
    {
        "tag":"name",
        "patterns":["What's your name?","who are you","whats ur name"],
        "responses":["My name is John","I'm John","John"]
        
        
    },
    {
        "tag":"goodbye",
        "patterns":["bye","later","cya"],
        "responses":["Bye","see you later"]
                                                    
    
    },
    {
        "tag": "date",
        "patterns":["what are you doing this weekend","do you want to hangout sometime?","what are your plans for this week"],
        "responses":["I am available all week","I don't have any plans","I am not busy"]
    },
    {
        "tag":"funny",
        "patterns":["Tell me a joke!","Tell me something funny!","Do you know a joke?"],
        "responses":["What did the buffalo say when his son left for college? Bison","Why did the hipster burn his mouth? He drank the coffee before it was cool."]
    }
      ]}

In [13]:
lm=WordNetLemmatizer() #for getting words
#lists
classes=[]
newWords=[]
doc_X=[]
doc_Y=[]
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens=nltk.word_tokenize(pattern)#tokenize the patterns
        newWords.extend(tokens)# extend the tokens
        doc_X.append(pattern)
        doc_Y.append(intent["tag"])
        
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
        
newWords=[lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] #setting words to lowercase if not in punctuation
newWords=sorted(set(newWords))
classes=sorted(set(classes))

In [15]:
print(classes
     )

['age', 'date', 'funny', 'goodbye', 'greeting', 'name']


In [16]:
print(newWords)

["'s", 'a', 'are', 'birthday', 'born', 'bye', 'cya', 'do', 'doing', 'for', 'funny', 'hangout', 'hello', 'hey', 'hi', 'how', 'is', 'joke', 'know', 'later', 'me', 'name', 'old', 'plan', 'something', 'sometime', 'tell', 'this', 'to', 'u', 'ur', 'want', 'week', 'weekend', 'were', 'what', 'whats', 'when', 'who', 'you', 'your']


In [17]:
print(doc_X)

['Hi', 'Hello', 'Hey', 'How are u?', 'how old are you', 'when were you born', 'when is your birthday', 'how old are u', "What's your name?", 'who are you', 'whats ur name', 'bye', 'later', 'cya', 'what are you doing this weekend', 'do you want to hangout sometime?', 'what are your plans for this week', 'Tell me a joke!', 'Tell me something funny!', 'Do you know a joke?']


In [18]:
print(doc_Y)

['greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'age', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'date', 'date', 'date', 'funny', 'funny', 'funny']


In [19]:
trainingdata=[] #training list array
outEmpty=[0]*len(classes)
for idx,doc in enumerate(doc_X):
    bagOfwords=[]
    text=lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)
    outputRow=list(outEmpty)
    outputRow[classes.index(doc_Y[idx])]=1
    trainingdata.append([bagOfwords,outputRow])
random.shuffle(trainingdata)
trainingdata=np.array(trainingdata,dtype=object)
train_x=np.array(list(trainingdata[:,0]))
train_y=np.array(list(trainingdata[:,1]))
   

In [20]:
ishape=(len(train_x[0]),)
oshape=len(train_y[0])
epochs=200
model=Sequential()
model.add(Dense(128,input_shape=ishape,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(oshape,activation="softmax"))
md=tf.keras.optimizers.Adam(learning_rate=0.01,decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
print(model.summary()) #output the model in summary
model.fit(x=train_x,y=train_y,epochs=200,verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5376      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 14,022
Trainable params: 14,022
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
1/1 [==============================

In [21]:
def ourtext(text):
   tokens=nltk.word_tokenize(text)
   tokens=[lm.lemmatize(word) for word in tokens]
   return tokens

def wordbag(text,vocab):
    tokens=ourtext(text)
    bagOfwords=[0]*len(vocab)
    for w in tokens:
        for idx,word in enumerate(vocab):
            if word==w:
                bagOfwords[idx]=1
    return np.array(bagOfwords)

def Pclass(text,vocab,labels):
    bagOfwords=wordbag(text,vocab)
    result=model.predict(np.array([bagOfwords]))[0]
    thresh=0.2
    yp=[[idx,res] for idx,res in enumerate(result) if res>thresh]
    yp.sort(key=lambda x:x[1],reverse=True)
    newlist=[]
    for r in yp:
        newlist.append(labels[r[0]])
    return newlist

def get_response(firstlist,firstJson):
    tag=firstlist[0]
    listofIntents=firstJson["intents"]
    for i in listofIntents:
        if i["tag"]==tag:
            result=random.choice(i["responses"])
            break
    return result

In [ ]:
while True:
    new_msg=input("")
    intents=Pclass(new_msg,newWords,classes)
    result=get_response(intents,data)
    print(result)

hello
Howdy partner!
how old are u
I am 20 years old
what's ur name
I'm John
Tell me something funny
Why did the hipster burn his mouth? He drank the coffee before it was cool.
What are you doing this weekend
I am available all week
bye
Bye
